In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# EDA:

keep in mind:
- Brown Dwarf -> Star Type = 0
- Red Dwarf -> Star Type = 1
- White Dwarf-> Star Type = 2
- Main Sequence -> Star Type = 3
- Supergiant -> Star Type = 4
- Hypergiant -> Star Type = 5

## Let`s download and see the data:

In [ ]:
data = pd.read_csv('/kaggle/input/star-dataset/6 class csv.csv')
data.head(10)

In [ ]:
print('shape: ', data.shape)

In [ ]:
data.describe()

Have we got well-balanced data?

In [ ]:
sns.set(style="darkgrid")
stars_types = pd.DataFrame(data['Star type'].value_counts().sort_values(ascending=False))
plt.figure(figsize=(15,5))
ax = sns.barplot(x = stars_types.index, y = 'Star type' , data = stars_types, palette='pastel')

Explore color types:

In [ ]:
stars_color = pd.DataFrame(data['Star color'].value_counts().sort_values(ascending=False))
plt.figure(figsize=(15,5))
ax = sns.barplot(x = stars_color.index, y = 'Star color' , data = stars_color, palette='pastel')
ax = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

Explore spectral classes:

In [ ]:
stars_spectral = pd.DataFrame(data['Spectral Class'].value_counts().sort_values(ascending=False))
plt.figure(figsize=(15,5))
ax = sns.barplot(x = stars_spectral.index, y = 'Spectral Class' , data = stars_spectral, palette='pastel')
ax = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

Have we got any linear dependences of our data?

Let`s explore this by correlation. 


In [ ]:
stars_data = {
    'temperature': data['Temperature (K)'],
    'luminosity': data['Luminosity(L/Lo)'],
    'radius': data['Radius(R/Ro)'],
    'absolute_magnitude': data['Absolute magnitude(Mv)'],
    'star_type': data['Star type'],
    'star_color': data['Star color'],
    'spectral_class': data['Spectral Class']
}
stars_data = pd.DataFrame.from_dict(stars_data)
stars_data['star_type'] = stars_data['star_type'].astype('category').cat.codes
stars_data['star_color'] = stars_data['star_color'].astype('category').cat.codes
stars_data['spectral_class'] = stars_data['spectral_class'].astype('category').cat.codes

corr = stars_data.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(10, 10))
cmap = sns.diverging_palette(200, 21, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()
corr

And we can see the most high dependenses between:
- temp and radius;
- star_type and star_color, spectral_class;
- star_color and radius, spectral_class.

Let`s see distibution of our numeric data:

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(10, 10))
sns.despine(left=True)

sns.distplot(stars_data['temperature'], color='b', ax=axes[0, 0])
sns.distplot(stars_data['luminosity'], color='m', ax=axes[0, 1])
sns.distplot(stars_data['radius'], color='r', ax=axes[1, 0])
sns.distplot(stars_data['absolute_magnitude'], color='g', ax=axes[1, 1])
plt.setp(axes, yticks=[])
plt.tight_layout()

Now see by boxplots, have we got outliers?

Exploring numeric features/star color. 

In [ ]:
ax = sns.catplot(x = 'Star color', y = 'Temperature (K)', kind = "box", data = data, palette='pastel')
ax = ax.fig.set_size_inches(30, 5)

In [ ]:
ax = sns.catplot(x = 'Star color', y = 'Luminosity(L/Lo)', kind = "box", data = data, palette='pastel')
ax = ax.fig.set_size_inches(30, 5)

In [ ]:
ax = sns.catplot(x = 'Star color', y = 'Radius(R/Ro)', kind = "box", data = data, palette='pastel')
ax = ax.fig.set_size_inches(30, 5)

In [ ]:
ax = sns.catplot(x = 'Star color', y = 'Absolute magnitude(Mv)', kind = "box", data = data, palette='pastel')
ax = ax.fig.set_size_inches(30, 5)

Let's add some interactive plots:

In [ ]:
import plotly.express as px

fig = px.scatter(data, x="Temperature (K)", y="Luminosity(L/Lo)", size="Radius(R/Ro)", color="Star color",
           hover_name="Star type", log_x=True, size_max=60)
fig.show()


Well, we have good data, because we have not a lot of outliers and our data are well-balanced.

# Classification. Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(stars_data.drop('star_type',axis=1), stars_data['star_type'], test_size=0.40, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)
predictions = dtree.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

confusion_matrix(y_test,predictions)

In [ ]:
print(classification_report(y_test,predictions))

Well, it is very good. Let`s try Logistic regression for this task.

# Classification. Logistic Regression

Try Logistic Regression with all features:

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=42, max_iter=10000)
logreg.fit(X_train, y_train)
predictions = logreg.predict(X_test)

print(classification_report(y_test, predictions))

Based on EDA, we can say that star color, spectral class have the greatest impact on star type:

In [ ]:
features_data = pd.DataFrame({'color': stars_data['star_color'], 'spectral_class': stars_data['spectral_class'], 'star_type': stars_data['star_type']})
X_train, X_test, y_train, y_test = train_test_split(features_data.drop('star_type',axis=1), features_data['star_type'], test_size=0.20, random_state=42)

In [ ]:
logreg.fit(X_train, y_train)
predictions = logreg.predict(X_test)

print(classification_report(y_test, predictions))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression


# Split data into training and test splits
train_idx, test_idx = train_test_split(stars_data.index, test_size=.2, random_state=42)
stars_data['split'] = 'train'
stars_data.loc[test_idx, 'split'] = 'test'

X = stars_data[['star_color', 'spectral_class']]
y = stars_data['star_type']
X_train = stars_data.loc[train_idx, ['star_color', 'spectral_class']]
y_train = stars_data.loc[train_idx, 'star_type']

# Condition the model on sepal width and length, predict the petal width
model = LinearRegression()
model.fit(X_train, y_train)
stars_data['prediction'] = model.predict(X)

fig = px.scatter(
    stars_data, x='star_color', y='spectral_class',
    marginal_x='histogram', marginal_y='histogram',
    color='split', trendline='ols'
)
fig.update_traces(histnorm='probability', selector={'type':'histogram'})
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=y.min(), y0=y.min(),
    x1=y.max(), y1=y.max()
)

fig.show()

Use only two features is bad idea, let`s add another features:

In [ ]:
features_data['temp'] = pd.Series(stars_data['temperature'], index = features_data.index)
features_data['radius'] = pd.Series(stars_data['radius'], index = features_data.index)

X_train, X_test, y_train, y_test = train_test_split(features_data.drop('star_type',axis=1), features_data['star_type'], test_size=0.20, random_state=42)
logreg.fit(X_train, y_train)
predictions = logreg.predict(X_test)

print(classification_report(y_test, predictions))

Look better, add luminosity:


In [ ]:
features_data['luminosity'] = pd.Series(stars_data['luminosity'], index = features_data.index)
X_train, X_test, y_train, y_test = train_test_split(features_data.drop('star_type', axis=1), features_data['star_type'], test_size=0.20, random_state=42)
logreg.fit(X_train, y_train)
predictions = logreg.predict(X_test)

print(classification_report(y_test, predictions))

So, quality has decreased.

In [ ]:
features_data['absolute_magnitude'] = pd.Series(stars_data['absolute_magnitude'], index = features_data.index)
del features_data['luminosity']

X_train, X_test, y_train, y_test = train_test_split(features_data.drop('star_type', axis=1), features_data['star_type'], test_size=0.20, random_state=42)
logreg.fit(X_train, y_train)
predictions = logreg.predict(X_test)

print(classification_report(y_test, predictions))

We see the quality has improved without using luminosity.